<a href="https://colab.research.google.com/github/mustufajp/Complete-Python-3-Bootcamp/blob/master/collaborative_recommender_cosine_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/SAAD/サード/Business Analysis/リコメンドシステム/Data Recomendation system/recomendation.csv')
df['product_code']=df['product_code'].fillna('')
df['product_category_code_type'] = df['product_category'].astype(str) + df['product_code'].astype(str)+df['product_type'].astype(str)
df_category=df[['customer_id','product_category','product_quantity']].groupby(['customer_id','product_category']).sum().reset_index()
df_item=df[['customer_id','sku_code_na_size','product_quantity']].groupby(['customer_id','sku_code_na_size']).sum().reset_index()


<ipython-input-6-eb9f124c0a40>:1: DtypeWarning: Columns (27,28,29,30,31,32,33,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/SAAD/サード/Business Analysis/リコメンドシステム/Data Recomendation system/recomendation.csv')


In [19]:
df_category.to_csv('product_category_bought_by_member.csv',index=False)

In [20]:
df_item.to_csv('product_bought_by_member.csv',index=False)

In [ ]:
df_recomend=pd.pivot_table(df_category, values='product_quantity', index=['customer_id'],
                       columns=['product_category'], aggfunc="sum",fill_value=0)
columns_to_keep_recomend=[col for col in df_recomend.columns if 'box' not in col]
df_user_product=df_recomend[columns_to_keep_recomend]
sparse_purchase_counts=sparse.csr_matrix(df_user_product)
cosine_similarities = cosine_similarity(sparse_purchase_counts.T)


In [ ]:
def recommend_items(user_id, n=2):
    # Get the user's purchase history
    user_history = sparse_purchase_counts[user_id].toarray().flatten()

    # Compute the average cosine similarity between the user's purchased items and all other items
    similarities = cosine_similarities.dot(user_history)

    # Get the indices of the user's purchased items
    purchased_indices = np.where(user_history > 0)[0]

    # Set the similarity scores for purchased items to 0
    similarities[purchased_indices] = 0

    # Sort the items by similarity score and return the top n items
    recommended_indices = np.argsort(similarities)[::-1][:n]
    recommended_items = list(df_user_product.columns[recommended_indices])

    return recommended_items

In [ ]:
recommend_items(123, n=3)

['HBP', 'bet', 'dbi']

In [ ]:
df_product_customer=pd.pivot_table(df_item, values='product_quantity', index=['customer_id'],
                       columns=['sku_code_na_size'], aggfunc="sum",fill_value=0)
columns_to_keep=[col for col in df_product_customer.columns if 'box' not in col]
df_products_na_box=df_product_customer[columns_to_keep]


In [ ]:
purchased_items=list(df_products_na_box.columns[df_products_na_box.loc[123] > 0])
recommended_items=[]

for i in recommend_items(123):
  top_item=[col for col in df_products_na_box.columns if col.startswith(i) and i not in purchased_items]

  df_recomended_items=df_item[df_item['sku_code_na_size'].isin(top_item)]
  highest_row=df_recomended_items.loc[df_recomended_items['product_quantity'].idxmax()]
  highest_item = highest_row['sku_code_na_size']
  recommended_items.append(highest_item)
print(recommended_items)

['HBP347S', 'bet50']


In [ ]:
df_product_quantity=df[['sku_code_na_size','product_quantity']].groupby(['sku_code_na_size']).sum().reset_index()
df_product_quantity

,sku_code_na_size,product_quantity
0,0,21
1,B105S,14
2,B109S,6
3,BET73,151
4,BET73SGPL,131
...,...,...
4939,xwt7s,26
4940,xwt8s,1
4941,z1,14782
4942,z2,5802
